# Optuna
+ 하이퍼파라미터 튜닝에 쓰고 있는 최신 Automl 기법입니다.
+ 빠르게 튜닝이 가능하다는 장점이 있습니다.
+ 하이퍼파라미터 튜닝 방식을 지정할수 있다. -> 직관적인 api인 튜닝된 lightgbm도 제공해줍니다.
+ 다른 라이브러리들에 비해 직관적인 장점이 있어 코딩하기 용이합니다.

In [1]:
import numpy as np
import pandas as pd
import optuna
from lightgbm import LGBMClassifier
from optuna import Trial
from optuna.samplers import TPESampler
from sklearn.metrics import log_loss
from sklearn.model_selection import train_test_split

# 간단한 전처리

In [2]:
path = "/kaggle/input/predictcreditcarddelinquency/"
train = pd.read_csv(path + "train.csv")
train = train.drop(["index"], axis=1)
train.fillna("NAN", inplace=True)

test = pd.read_csv(path + "test.csv")
test = test.drop(["index"], axis=1)
test.fillna("NAN", inplace=True)

In [3]:
train_ohe = pd.get_dummies(train)
test_ohe = pd.get_dummies(test)

In [4]:
X = train_ohe.drop(["credit"], axis=1)
y = train["credit"]
X_test = test_ohe.copy()

+ Optuna는 objective하이퍼 파라미터의 성능을 평가하고 향후 시험에서 샘플링 할 위치를 결정하기 위해 숫자 값을 반환 하는 함수가 필요하다는 것을 의미하는 블랙 박스 최적화 프로그램 입니다.
+ Optuna의 특정 인수를 object에 전달됩니다.
+ trial는 조정해야하는 하이퍼 파라미터를 지정 하기 위해 objective 함수에 전달됩니다.
+ 이것은 logloss 성능에 대한 피드백으로 Optuna에서 사용하는 모델에서 반환합니다.

In [5]:
def objective(trial: Trial) -> float:
    params_lgb = {
        "random_state": 42,
        "verbosity": -1,
        "learning_rate": 0.05,
        "n_estimators": 10000,
        "objective": "multiclass",
        "metric": "multi_logloss",
        "reg_alpha": trial.suggest_float("reg_alpha", 1e-8, 3e-5),
        "reg_lambda": trial.suggest_float("reg_lambda", 1e-8, 9e-2),
        "max_depth": trial.suggest_int("max_depth", 1, 20),
        "num_leaves": trial.suggest_int("num_leaves", 2, 256),
        "colsample_bytree": trial.suggest_float("colsample_bytree", 0.4, 1.0),
        "subsample": trial.suggest_float("subsample", 0.3, 1.0),
        "subsample_freq": trial.suggest_int("subsample_freq", 1, 10),
        "min_child_samples": trial.suggest_int("min_child_samples", 5, 100),
        "max_bin": trial.suggest_int("max_bin", 200, 500),
    }
    
    X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.2)

    model = LGBMClassifier(**params_lgb)
    model.fit(
        X_train,
        y_train,
        eval_set=[(X_train, y_train), (X_valid, y_valid)],
        early_stopping_rounds=100,
        verbose=False,
    )

    lgb_pred = model.predict_proba(X_valid)
    log_score = log_loss(y_valid, lgb_pred)
    
    return log_score

In [6]:
sampler = TPESampler(seed=42)
study = optuna.create_study(
    study_name="lgbm_parameter_opt",
    direction="minimize",
    sampler=sampler,
)
study.optimize(objective, n_trials=10)
print("Best Score:", study.best_value)
print("Best trial:", study.best_trial.params)

[I 2021-05-19 04:25:32,346] A new study created in memory with name: lgbm_parameter_opt
[I 2021-05-19 04:25:43,086] Trial 0 finished with value: 0.748023159446636 and parameters: {'reg_alpha': 1.12424581642324e-05, 'reg_lambda': 0.08556428806974939, 'max_depth': 11, 'num_leaves': 73, 'colsample_bytree': 0.759195090518222, 'subsample': 0.40921304830970556, 'subsample_freq': 3, 'min_child_samples': 91, 'max_bin': 287}. Best is trial 0 with value: 0.748023159446636.
[I 2021-05-19 04:26:02,527] Trial 1 finished with value: 0.7638953631593801 and parameters: {'reg_alpha': 1.0017921248059264e-05, 'reg_lambda': 0.01285802218430649, 'max_depth': 3, 'num_leaves': 151, 'colsample_bytree': 0.4338469474162602, 'subsample': 0.8053991405867773, 'subsample_freq': 6, 'min_child_samples': 6, 'max_bin': 391}. Best is trial 0 with value: 0.748023159446636.
[I 2021-05-19 04:26:15,979] Trial 2 finished with value: 0.7371244115293755 and parameters: {'reg_alpha': 2.9766424663143615e-05, 'reg_lambda': 0.0555

Best Score: 0.7209297492545786
Best trial: {'reg_alpha': 1.0997191680377813e-05, 'reg_lambda': 0.04104630401883339, 'max_depth': 15, 'num_leaves': 191, 'colsample_bytree': 0.5198042692950159, 'subsample': 0.6599641068895281, 'subsample_freq': 9, 'min_child_samples': 7, 'max_bin': 334}


In [7]:
# 시각화
optuna.visualization.plot_optimization_history(study)

In [8]:
# 파라미터들관의 관계
optuna.visualization.plot_parallel_coordinate(study)

In [9]:
# 각 파라미터들의 상관관계
optuna.visualization.plot_contour(
    study,
    params=[
        "max_depth",
        "num_leaves",
        "colsample_bytree",
        "subsample",
        "subsample_freq",
        "min_child_samples",
        "max_bin",
    ],
)

In [10]:
# 하이퍼파라미터 중요도
optuna.visualization.plot_param_importances(study)